In [1]:
import matplotlib.pylab as plt
import numpy as np
import xarray as xr
import pandas as pd
from scipy import signal
import glob

In [34]:
from tools_pcc import *

In [35]:
from xdas.io.febus import read as read_das
from xdas.io.febus import correct_gps_time
from stack_functions_new import *

In [36]:
user = "mba"

if user == "lucie" :
    path1 = "/home/invite/Documents/DAS-Lucie/fichiers-nc/"

elif user == "mba" :
    path1 = "/Volumes/DISKENOIR/fichiers-nc/"
    path2 = "/Volumes/DISKENOIR/Correlograms/Cross-correlation"

Cross-correlation

In [10]:
# define variable for correlation
window = 300 			# duration of time window in second
lag = window/2 			# shift of starting time on the next iteration
time = 0 			# time initialization

# filter boundary (Hz)
lowcut = 1
highcut = 10
fs = 50 # Hz

start = 100		# starting/reference point. currently used: 100, 200, 320, 400
channel = 150		# number of channel used			 
start_position = 'mid_all' 

folder_destination = path2 # please make the folder before running

change_gauge_length = False
gauge_length = 50

In [11]:
dt = 0.02

In [12]:
# make output array
if start_position == 'mid':
	cc_normal = np.zeros((2*int(window/dt),2*channel+1))
	cc_onebit = np.zeros((2*int(window/dt),2*channel+1))
	cc_phase = np.zeros((2*int(window/dt),2*channel+1))
elif start_position == 'mid_all':
	cc_normal = np.zeros((2*int(window/dt),430+1))
	cc_onebit = np.zeros((2*int(window/dt),430+1))
	cc_phase = np.zeros((2*int(window/dt),430+1)) # change to 100-57+1 if only want from 57 to 100
else:
	cc_normal = np.zeros((2*int(window/dt),channel+1))
	cc_onebit = np.zeros((2*int(window/dt),channel+1))
	cc_phase = np.zeros((2*int(window/dt),channel+1))

In [13]:
f = open('error.txt','w') # make file containing files with few channels

In [14]:
ds = xr.open_dataset(path1 + "SR_FEBUS_STROMBOLI_2022-09-18_17-35-44_UTC.nc")
array=ds['strain_rates']

In [15]:
ds

<xarray.Dataset>
Dimensions:       (times: 180000, offsets: 3250, channels: 3250)
Coordinates:
  * times         (times) datetime64[ns] 2022-09-18T17:35:44.500000 ... 2022-...
  * offsets       (offsets) float64 0.0 2.0 4.0 ... 6.496e+03 6.498e+03
  * channels      (channels) float64 0.0 1.0 2.0 ... 3.248e+03 3.249e+03
Data variables:
    strain_rates  (times, offsets) float32 ...
Attributes:
    unit:     strain rates in 10e-8 /s

In [16]:
array

<xarray.DataArray 'strain_rates' (times: 180000, offsets: 3250)>
[585000000 values with dtype=float32]
Coordinates:
  * times    (times) datetime64[ns] 2022-09-18T17:35:44.500000 ... 2022-09-18...
  * offsets  (offsets) float64 0.0 2.0 4.0 6.0 ... 6.494e+03 6.496e+03 6.498e+03

In [17]:
channels_coord = ds['channels']

sr_array_with_channels = xr.DataArray(
    data=ds['strain_rates'],  # Remplacez par les données réelles de 'strain_rates'
    dims=("times", "channels"),  # Spécifiez les dimensions
    coords={"times": ds['strain_rates'].coords["times"], "channels": channels_coord},
)

In [18]:
sr_array_with_channels

<xarray.DataArray 'strain_rates' (times: 180000, channels: 3250)>
array([[ 0.0000000e+00,  0.0000000e+00, -3.4953697e-08, ...,
         4.4692129e+03, -8.0042388e-05,  5.1425472e-05],
       [ 0.0000000e+00,  0.0000000e+00,  6.1442131e-09, ...,
         2.8822549e+03, -5.6834502e-05,  3.3164910e-05],
       [ 0.0000000e+00,  0.0000000e+00, -1.7207492e-08, ...,
         3.8322634e+03, -7.1476410e-05,  4.4096316e-05],
       ...,
       [ 0.0000000e+00, -8.2707105e-12,  1.3172808e-08, ...,
         1.0095570e+02, -1.7541629e-06,  1.1616555e-06],
       [ 0.0000000e+00,  0.0000000e+00,  7.6554487e-09, ...,
         9.9110550e+01, -1.7135526e-06,  1.1404245e-06],
       [ 0.0000000e+00,  0.0000000e+00, -3.6883813e-10, ...,
         9.9572304e+01, -1.7854875e-06,  1.1457379e-06]], dtype=float32)
Coordinates:
  * times     (times) datetime64[ns] 2022-09-18T17:35:44.500000 ... 2022-09-1...
  * channels  (channels) float64 0.0 1.0 2.0 ... 3.247e+03 3.248e+03 3.249e+03

In [19]:
iDas_nc = sr_array_with_channels
Das_nc = sr_array_with_channels.coords

In [38]:
iDas_nc

<xarray.DataArray 'strain_rates' (times: 180000, channels: 3250)>
array([[ 0.0000000e+00,  0.0000000e+00, -3.4953697e-08, ...,
         4.4692129e+03, -8.0042388e-05,  5.1425472e-05],
       [ 0.0000000e+00,  0.0000000e+00,  6.1442131e-09, ...,
         2.8822549e+03, -5.6834502e-05,  3.3164910e-05],
       [ 0.0000000e+00,  0.0000000e+00, -1.7207492e-08, ...,
         3.8322634e+03, -7.1476410e-05,  4.4096316e-05],
       ...,
       [ 0.0000000e+00, -8.2707105e-12,  1.3172808e-08, ...,
         1.0095570e+02, -1.7541629e-06,  1.1616555e-06],
       [ 0.0000000e+00,  0.0000000e+00,  7.6554487e-09, ...,
         9.9110550e+01, -1.7135526e-06,  1.1404245e-06],
       [ 0.0000000e+00,  0.0000000e+00, -3.6883813e-10, ...,
         9.9572304e+01, -1.7854875e-06,  1.1457379e-06]], dtype=float32)
Coordinates:
  * times     (times) datetime64[ns] 2022-09-18T17:35:44.500000 ... 2022-09-1...
  * channels  (channels) float64 0.0 1.0 2.0 ... 3.247e+03 3.248e+03 3.249e+03

In [20]:
Das_nc

Coordinates:
  * times     (times) datetime64[ns] 2022-09-18T17:35:44.500000 ... 2022-09-1...
  * channels  (channels) float64 0.0 1.0 2.0 ... 3.247e+03 3.248e+03 3.249e+03

In [26]:
iDas_nc[:,3]

<xarray.DataArray 'strain_rates' (times: 180000)>
array([-5.4988828e-07, -4.4288828e-07,  4.1325757e-06, ...,
       -7.1864732e-04,  4.7920668e-08,  6.3239902e-08], dtype=float32)
Coordinates:
  * times     (times) datetime64[ns] 2022-09-18T17:35:44.500000 ... 2022-09-1...
    channels  float64 3.0

In [48]:
if iDas_nc.shape[1] < 463: 
    print(f'This file only has {iDas_nc.shape[1]} channels. Skipping this file')
    f.write(f'{iDas_nc.shape[1]} channels\n')
	# skipping file if the channel is not complete

else :

    print(f'Filtering using bandpass filter between {lowcut} and {highcut} Hz ...')

    if change_gauge_length == False:
        traces_filt = Das_nc.copy()
    else:
        
        # to try GL = 10
        traces_filt = iDas_nc.copy()
        traces_filt['strain_rates'][:,0] = Das_nc['strain_rates'][:,0] + Das_nc['strain_rates'][:,1] # the first channel with gl = 10 is defined as channel 0 + channel 1
        traces_filt['strain_rates'][:,-1] = Das_nc['strain_rates'][:,-2] + Das_nc['strain_rates'][:,-1] # the last channel with gl = 10 is defined as channel N-1 + channel N
        print(f'Converting data to using gauge length = 10')
        #for i in range(1,traces_filt.shape[1]-2): # each end is skipped because it has been calculated in the previous line
        for i_channel in range(57, Das_nc['channels'].values[-2]+1):
            traces_filt['strain_rates'][:,i_channel] = Das_nc['strain_rates'][:,i_channel-1] + Das_nc['strain_rates'][:,i_channel] + Das_nc['strain_rates'][:,i_channel+1]
        traces_filt_gl10 = traces_filt['strain_rates'][::2]

## à modifier ??? c'est les channels qu'on veut modifier ou les strain rates ? (au-dessus)

    for j in [57,430]:
        try:
            #traces_filt[:,j] = filter_bandpass(Das_nc[:,j], lowcut, highcut, fs,order=4) # scipy.signal filter
            traces_filt[:,j] = bandpass(Das_nc[:,j],lowcut,highcut,fs, zerophase=True) # obspy filter (faster)
        except:
            None
		
    length = iDas_nc.shape[0]	# total length of data

	# cross correlate

    print('\nStarting the phase auto-correlation  ...')
    cc_phase_auto, t = auto_correlate(cc_phase,traces_filt,folder_destination,window,lag,length,time,channel,
            start,start_position,onebit=False,phase_cc=True,decimate=1,resample=1)

    #print('\nStarting the phase cross-correlation  ...')
    #cc_phase, t = cross_correlate(cc_phase,traces_filt,folder_des;tination,window,lag,length,time,channel,
    #		start,start_position,onebit=False,phase_cc=True,decimate=1,resample=1)

    #print('\nStarting the normal cross-correlation  ...')
    #cc_normal, t = cross_correlate(cc_normal,traces_filt,window,lag,length,time,channel,
    #		start,start_position,onebit=False,phase_cc=False,decimate=decimate,resample=resample)

    #cc_onebit, t = cross_correlate(cc_onebit,traces_filt,window,lag,length,time,channel,
    #		start,start_position,onebit=True,phase_cc=False,decimate=decimate,resample=resample)



f.close()

Filtering using bandpass filter between 1 and 10 Hz ...

Starting the phase auto-correlation  ...


KeyError: (1, 200)

In [46]:
traces_filt

Coordinates:
  * times     (times) datetime64[ns] 2022-09-18T17:35:44.500000 ... 2022-09-1...
  * channels  (channels) float64 0.0 1.0 2.0 ... 3.247e+03 3.248e+03 3.249e+03

In [47]:
Das_nc['strain_rates'][:,0] + Das_nc['strain_rates'][:,1]

KeyError: 'strain_rates'

In [ ]:
f.close()